In [ ]:
#data
docs = ['Where are you REALLY from?',
       'Go back to where you came from!',
       'What\'s my Chinese name?',
       'Are you from North or South Korea?',
       'They\'re so pretty for a black person',
       'Scientifically speaking, whites are superior to blacks',
       'Filipinos, Mexicans, they\'re all the same!',
       'I bet I know Japanese better than you lol',
       'You\re such a banana',
       'You don\'t act black at all, in fact I just think of you as white!',
       'Go find a god damned job and learn to speak English or I\'ll blind you with a piece of dental floss, you slanty-eyed mother fucker',
        'Y\'all are like the good kind of black people!',
       'That Jew belongs in an oven.',
        'She needs to get her Jap ass out.',
       'I love you so much, you\'re so cool!!!',
       'She\'s such a hard worker, and I really appreciate that.',
       'The state of this nation is a travesty and something needs to change.',
       'Obama was the greatest president this country has seen.',
       'Machine learning is a method of data analysis that automates analytical model building. It is a branch of artificial intelligence based on the idea that systems can learn from data, identify patterns and make decisions with minimal human intervention.',
       'I\'d like a nice bowl of ramen to hit the spot right now!',
       'Capitalism is an unsustainable and unjust idea, and we need to make changes before global warming destroys the world.',
       'The cutest member of BTS has to be V, but if we\'re going off sense of humor, Jin. However, if we\'re considering intellect, RM. Not that you asked but the best dancer is JHOPE and the guy I can\'t believe is my age is Jungkook.',
       'For people who have red-green colorblindness, the colors red and green look exactly the same.',
       'We hope to see you again!']

#labels
labels = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0])

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import Callback
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

import pandas as pd
import numpy as np

In [27]:
#hyperparameters
max_features = 5000
max_length = 30

In [52]:
#split the data into train, test, and validation sets
data = pd.read_csv('dennisFiles/data/duplicated_modified_davidson_data.csv', names = ['tweet', 'label'])
train_portion = data.shape[0]*0.64
test_portion = train_portion + data.shape[0]*0.2

train = data.loc[0:train_portion,]
test = data.loc[train_portion:test_portion,]
valid = data.loc[test_portion:,]

#x_train = train.ix[0:train_portion,0].values
#y_train = train.ix[0:train_portion,1].values

#x_test = train.ix[train_portion:test_portion, 0].values
#y_test = train.ix[train_portion:test_portion, 1].values

#x_valid = train.ix[test_portion:,0].values
#y_valid = train.ix[test_portion:,1].values

                                                   tweet  label
0      !!! RT @mayasolovely: As a woman you shouldn't...      0
1      RT @theChefBoyRD: @victoriaaa_lee @anniespring...      1
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...      0
3                                @SexWith_SocksOn faggot      1
4      @karmendanielle_ u gotta slow down these coon ...      1
5      !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...      0
6                          @ViewFromBoxSeat She's a cunt      1
7      !!!!&#8220;@selfiequeenbri: cause I'm tired of...      0
8      "@ashlingwilde: @ItsNotAdam is bored supposed ...      1
9      &#8220;@Crowdifornia: We agree... do you? http...      1
10     " Keeks is a bitch she curves everyone " lol I...      0
11     Happy Columbus Day! Even #BillMaher likes #coc...      1
12     @DavetheBrave1 that hoe on some other shit bro...      1
13            RT @PeterpandaCat: @savanigga Sav is a fag      1
14                               " bitch

In [35]:
#preprocessing

In [46]:
#encode and pad
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['tweet'])

#encoded_train = [one_hot(word, max_features) for word in train]
#padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
sequences = tokenizer.texts_to_sequences(train['tweet'])
train_data = pad_sequences(sequences, maxlen=max_length, padding='post')

In [47]:
#changing class weights
#class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
#class_weights = {0: 1.,
                 #1: 40.}

#glove embeddings
embeddings_index = dict()
f = open('../Glove-1.2/vocab.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((max_features, 100))
for word, index in tokenizer.word_index.items():
    if index > max_features - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [48]:
#shape
model = Sequential()
model.add(Embedding(max_features, output_dim=100, weights=[embedding_matrix]))
model.add(LSTM(128))
model.add(Dense(64, activation='sigmoid'))
#model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [53]:
#compile
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#fit
model.fit(train_data, train['label'], batch_size=2048, epochs=15)
#score = model.evaluate(padded_test, y_test, batch_size=96)

Epoch 1/15
15862/15862 [==============================] - 8s 497us/step - loss: 0.4345 - acc: 0.8118
Epoch 2/15
15862/15862 [==============================] - 6s 374us/step - loss: 0.3593 - acc: 0.8560
Epoch 3/15
15862/15862 [==============================] - 6s 363us/step - loss: 0.3478 - acc: 0.8577
Epoch 4/15
15862/15862 [==============================] - 6s 356us/step - loss: 0.3486 - acc: 0.8578
Epoch 5/15
15862/15862 [==============================] - 6s 367us/step - loss: 0.5212 - acc: 0.7904
Epoch 6/15
15862/15862 [==============================] - 6s 368us/step - loss: 0.3410 - acc: 0.8613
Epoch 7/15
15862/15862 [==============================] - 6s 384us/step - loss: 0.3383 - acc: 0.8624
Epoch 8/15
15862/15862 [==============================] - 6s 384us/step - loss: 0.3416 - acc: 0.8585
Epoch 9/15
15862/15862 [==============================] - 6s 391us/step - loss: 0.3569 - acc: 0.8548
Epoch 10/15
15862/15862 [==============================] - 6s 369us/step - loss: 0.3139 - a

In [54]:
#test = pd.read_csv('dennisFiles/data/expert_labeled_tweets_duplicate.csv')

#x_test = test.ix[:, 0].values
#y_test = test.ix[:, 1].values

#encoded_test = t.texts_to_matrix(x_test, mode='count')
#encoded_test = [hashing_trick(word, len(text_to_word_sequence(word))*1.3, hash_function='md5') for word in x_test]
sequences = tokenizer.texts_to_sequences(test['tweet'])
test_data = pad_sequences(sequences, maxlen=max_length, padding='post')

pred = [1 if element >= 0.5 else 0 for element in model.predict(test_data)]
print(classification_report(test['label'], pred))

             precision    recall  f1-score   support

          0       0.98      0.94      0.96      4764
          1       0.25      0.51      0.34       192

avg / total       0.95      0.92      0.94      4956

